## Import statements

In [18]:
import json
import pandas as pd
import zipfile
from IPython.display import display

## Data preparation

### 1. Weather data

In [19]:
with open('weather/2024-01-01.json') as f:
    data = json.load(f)

def convert_utc_datetime(df: pd.DataFrame, column: str):
    # Parse as datetime with timezone handling
    df[column] = pd.to_datetime(df[column], format='mixed', utc=True)

    # Convert to ISO UTC string with Z
    df[column] = df[column].dt.strftime('%Y-%m-%dT%H:%M:%SZ')

# Flatten JSON
weather = pd.json_normalize(data)
convert_utc_datetime(weather, 'properties.from')
convert_utc_datetime(weather, 'properties.created')
convert_utc_datetime(weather, 'properties.calculatedAt')
convert_utc_datetime(weather, 'properties.to')

display(weather)

,type,id,geometry.coordinates,geometry.type,properties.calculatedAt,properties.created,properties.from,properties.parameterId,properties.qcStatus,properties.stationId,properties.timeResolution,properties.to,properties.validity,properties.value,properties.noValuesInCalculation,properties.cloudCoverMethod,properties.extremeOccurence.type,properties.extremeOccurence.value
0,Feature,001f83a3-5ef7-3106-86bb-ddbdef4142bd,"[12.4121, 55.8764]",Point,2024-01-03T03:03:13Z,2024-01-03T14:14:34Z,2024-01-01T23:00:00Z,mean_relative_hum,manual,06188,hour,2024-01-02T00:00:00Z,True,93.5,NaN,NaN,NaN,NaN
1,Feature,006c4afa-2207-b54a-75b3-b9af8d74fa13,"[9.1674, 55.7379]",Point,2024-01-04T08:42:19Z,2024-01-04T08:42:20Z,2024-01-01T23:00:00Z,max_wind_speed_3sec,manual,06104,day,2024-01-02T23:00:00Z,True,21.6,24.0,NaN,NaN,NaN
2,Feature,00743083-2830-f046-5440-bfa86b3bf1d7,"[9.5067, 56.7558]",Point,2024-01-03T03:03:14Z,2024-01-03T14:14:54Z,2024-01-01T23:00:00Z,max_wind_speed_10min,manual,06065,hour,2024-01-02T00:00:00Z,True,8.5,NaN,NaN,NaN,NaN
3,Feature,00952cb4-3797-4392-5608-d4ccf1ab01ce,"[8.6705, 56.383]",Point,2024-01-03T07:23:16Z,2024-01-03T14:15:17Z,2024-01-01T23:00:00Z,bright_sunshine,manual,06056,day,2024-01-02T23:00:00Z,True,0.0,24.0,NaN,NaN,NaN
4,Feature,009a06ea-531d-2993-c75c-d83cdae9a92e,"[-52.6961, 70.7342]",Point,2024-01-02T00:50:30Z,2025-05-16T10:24:49Z,2024-01-01T23:00:00Z,max_wind_speed_3sec,manual,04213,hour,2024-01-02T00:00:00Z,True,9.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29390,Feature,fe3786ea-eb14-a9a3-186f-a45b836d5cfb,"[-37.6367, 65.6111]",Point,2024-01-01T01:51:23Z,2024-01-02T13:21:56Z,2024-01-01T00:00:00Z,mean_radiation,manual,04360,hour,2024-01-01T01:00:00Z,True,0.0,NaN,NaN,NaN,NaN
29391,Feature,feb7d1bf-b166-5aaf-11cd-12827997c9b4,"[8.3207, 56.7637]",Point,2024-01-02T15:04:31Z,2024-01-03T09:47:17Z,2024-01-01T00:00:00Z,min_temp,manual,06051,hour,2024-01-01T01:00:00Z,True,5.1,NaN,NaN,NaN,NaN
29392,Feature,fedc163e-4573-6d55-e70a-cc8a45b6e596,"[12.3424, 56.1193]",Point,2024-01-02T15:04:36Z,2024-01-03T09:47:38Z,2024-01-01T00:00:00Z,mean_pressure,manual,06168,hour,2024-01-01T01:00:00Z,True,1001.0,NaN,NaN,NaN,NaN
29393,Feature,ff472913-b6f4-5f1f-093a-43ef445ab00e,"[-42.0678, 61.9364]",Point,2024-01-01T01:51:23Z,2025-05-16T10:23:53Z,2024-01-01T00:00:00Z,mean_wind_dir,manual,04382,hour,2024-01-01T01:00:00Z,True,347.0,NaN,NaN,NaN,NaN


### 2. Day ahead prices

In [20]:
day_ahead = pd.read_csv('DayAheadPrices/DayAheadPrices_DK1_202401010000-202501010000.csv')

day_ahead['MTU (UTC)'] = day_ahead['MTU (UTC)'].str.split(' - ').str[1]
day_ahead['MTU (UTC)'] = pd.to_datetime(day_ahead['MTU (UTC)'], format='%d/%m/%Y %H:%M:%S')
day_ahead['MTU (UTC)'] = day_ahead['MTU (UTC)'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
display(day_ahead)

,MTU (UTC),Area,Sequence,Day-ahead Price (EUR/MWh),Intraday Period (UTC),Intraday Price (EUR/MWh)
0,2024-01-01T00:00:00Z,BZN|DK1,Without Sequence,16.99,NaN,NaN
1,2024-01-01T01:00:00Z,BZN|DK1,Without Sequence,28.14,NaN,NaN
2,2024-01-01T02:00:00Z,BZN|DK1,Without Sequence,26.66,NaN,NaN
3,2024-01-01T03:00:00Z,BZN|DK1,Without Sequence,4.14,NaN,NaN
4,2024-01-01T04:00:00Z,BZN|DK1,Without Sequence,-0.03,NaN,NaN
...,...,...,...,...,...,...
8779,2024-12-31T19:00:00Z,BZN|DK1,Without Sequence,25.02,NaN,NaN
8780,2024-12-31T20:00:00Z,BZN|DK1,Without Sequence,19.00,NaN,NaN
8781,2024-12-31T21:00:00Z,BZN|DK1,Without Sequence,15.70,NaN,NaN
8782,2024-12-31T22:00:00Z,BZN|DK1,Without Sequence,9.06,NaN,NaN


### 3. Consumption

In [21]:
with zipfile.ZipFile('2024_Consumption.zip') as z:
    # Get the first (and only) file inside the zip
    file_name = z.namelist()[0]

    # Open the file inside the zip and read it with pandas
    with z.open(file_name) as f:
        consumption = pd.read_csv(f)

consumption['TimeUTC'] = pd.to_datetime(consumption['TimeUTC'], format='%Y-%m-%dT%H:%M:%S')
consumption['TimeUTC'] = consumption['TimeUTC'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
display(consumption)

,TimeUTC,TimeDK,MunicipalityCode,Municipality,RegionName,HousingCategory,HeatingCategory,ConsumptionkWh
0,2023-12-31T23:00:00Z,2024-01-01T00:00:00,860,Hjørring,Region Nordjylland,Parcel- og rækkehuse,Elvarme eller varmepumpe,5053.475
1,2023-12-31T23:00:00Z,2024-01-01T00:00:00,860,Hjørring,Region Nordjylland,Parcel- og rækkehuse,Andet,7957.491
2,2023-12-31T23:00:00Z,2024-01-01T00:00:00,860,Hjørring,Region Nordjylland,Fritidshuse,Elvarme eller varmepumpe,3813.127
3,2023-12-31T23:00:00Z,2024-01-01T00:00:00,860,Hjørring,Region Nordjylland,Fritidshuse,Andet,218.168
4,2023-12-31T23:00:00Z,2024-01-01T00:00:00,860,Hjørring,Region Nordjylland,Etageejendom,Elvarme eller varmepumpe,89.426
...,...,...,...,...,...,...,...,...
5699726,2024-12-31T22:00:00Z,2024-12-31T23:00:00,101,København,Region Hovedstaden,Fritidshuse,Andet,178.665
5699727,2024-12-31T22:00:00Z,2024-12-31T23:00:00,101,København,Region Hovedstaden,Etageejendom,Elvarme eller varmepumpe,514.990
5699728,2024-12-31T22:00:00Z,2024-12-31T23:00:00,101,København,Region Hovedstaden,Etageejendom,Andet,42063.577
5699729,2024-12-31T22:00:00Z,2024-12-31T23:00:00,101,København,Region Hovedstaden,Andet,Elvarme eller varmepumpe,30.190
